# Getting started with machine learning pipelines
>  PySpark has built-in, cutting-edge machine learning routines, along with utilities to create full machine learning pipelines. You'll learn about them in this chapter.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 3 exercises "Introduction to PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [4]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

## Machine Learning Pipelines

<div class=""><p>In the next two chapters you'll step through every stage of the machine learning pipeline, from data intake to model evaluation. Let's get to it!</p>
<p>At the core of the <code>pyspark.ml</code> module are the <code>Transformer</code> and <code>Estimator</code> classes. Almost every other class in the module behaves similarly to these two basic classes.</p>
<p><code>Transformer</code> classes have a <code>.transform()</code> method that takes a DataFrame and returns a new DataFrame; usually the original one with a new column appended. For example, you might use the class <code>Bucketizer</code> to create discrete bins from a continuous feature or the class <code>PCA</code> to reduce the dimensionality of your dataset using principal component analysis.</p>
<p><code>Estimator</code> classes all implement a <code>.fit()</code> method. These methods also take a DataFrame, but instead of returning another DataFrame they return a model object. This can be something like a <code>StringIndexerModel</code> for including categorical data saved as strings in your models, or a <code>RandomForestModel</code> that uses the random forest algorithm for classification or regression.</p>
<hr>
<p>Which of the following is <strong>not</strong> true about machine learning in Spark?</p></div>

<pre>
Possible Answers
<b>Spark's algorithms give better results than other algorithms.</b>
Working in Spark allows you to create reproducible machine learning pipelines.
Machine learning pipelines in Spark are made up of Transformers and Estimators.
PySpark uses the pyspark.ml submodule to interface with Spark's machine learning routines.
</pre>

**Spark is just a platform that implements the same algorithms that can be found elsewhere.**

## Join the DataFrames

<p>In the next two chapters you'll be working to build a model that predicts whether or not a flight will be delayed based on the flights data we've been working with. This model will also include information about the plane that flew the route, so the first step is to join the two tables: <code>flights</code> and <code>planes</code>!</p>

In [13]:
%%capture
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/flights_small.csv
spark = (SparkSession.builder.appName("flights").getOrCreate())
flights = (spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("flights_small.csv"))
flights.createOrReplaceTempView("flights")

!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/planes.csv
planes = (spark.read.format("csv").option("inferSchema", "true").option("header", "true").load('planes.csv'))
planes.createOrReplaceTempView("planes")

Instructions
<ul>
<li>First, rename the <code>year</code> column of <code>planes</code> to <code>plane_year</code> to avoid duplicate column names.</li>
<li>Create a new DataFrame called <code>model_data</code> by joining the <code>flights</code> table with <code>planes</code> using the <code>tailnum</code> column as the key.</li>
</ul>

In [14]:
# Rename year column
planes = planes.withColumnRenamed("year", "plane_year")

# Join the DataFrames
model_data = flights.join(planes, on='tailnum', how="leftouter")

## Data types

<div class=""><p>Good work! Before you get started modeling, it's important to know that Spark only handles numeric data. That means all of the columns in your DataFrame must be either integers or decimals (called 'doubles' in Spark).</p>
<p>When we imported our data, we let Spark guess what kind of information each column held. Unfortunately, Spark doesn't always guess right and you can see that some of the columns in our DataFrame are strings containing numbers as opposed to actual numeric values.</p>
<p>To remedy this, you can use the <code>.cast()</code> method in combination with the <code>.withColumn()</code> method.
It's important to note that <code>.cast()</code> works on columns, while <code>.withColumn()</code> works on DataFrames.</p>
<p>The only argument you need to pass to <code>.cast()</code> is the kind of value you want to create, in string form. For example, to create integers, you'll pass the argument <code>"integer"</code> and for decimal numbers you'll use <code>"double"</code>.</p>
<p>You can put this call to <code>.cast()</code> inside a call to <code>.withColumn()</code> to overwrite the already existing column, just like you did in the previous chapter!</p>
<hr>
<p>What kind of data does Spark need for modeling?</p></div>

<pre>
Possible Answers
Doubles
Integers
Decimals
<b>Numeric</b>
Strings
</pre>

**Spark needs numeric values (doubles or integers) to do machine learning.**

## String to integer

<div class=""><p>Now you'll use the <code>.cast()</code> method you learned in the previous exercise to convert all the appropriate columns from your DataFrame <code>model_data</code> to integers!</p>
<p>To convert the type of a column using the <code>.cast()</code> method, you can write code like this:</p>
<pre><code>dataframe = dataframe.withColumn("col", dataframe.col.cast("new_type"))
</code></pre></div>

Instructions
<li>Use the method <code>.withColumn()</code> to <code>.cast()</code> the following columns to type <code>"integer"</code>. Access the columns using the <code>df.col</code> notation:<ul>
<li><code>model_data.arr_delay</code></li>
<li><code>model_data.air_time</code></li>
<li><code>model_data.month</code></li>
<li><code>model_data.plane_year</code></li></ul></li>

In [38]:
# Cast the columns to integers
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn("air_time", model_data.air_time.cast('integer'))
model_data = model_data.withColumn("month", model_data.month.cast('integer'))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast('integer'))

## Create a new column

<p>In the last exercise, you converted the column <code>plane_year</code> to an integer. This column holds the year each plane was manufactured. However, your model will use the planes' <em>age</em>, which is slightly different from the year it was made!</p>

Instructions
<li>Create the column <code>plane_age</code> using the <code>.withColumn()</code> method and subtracting the year of manufacture (column <code>plane_year</code>) from the year (column <code>year</code>) of the flight.</li>

In [39]:
# Create the column plane_age
model_data = model_data.withColumn("plane_age", model_data.year - model_data.plane_year)

## Making a Boolean

<p>Consider that you're modeling a yes or no question: is the flight late? However, your data contains the arrival delay in minutes for each flight. Thus, you'll need to create a boolean column which indicates whether the flight was late or not!</p>

Instructions
<ul>
<li>Use the <code>.withColumn()</code> method to create the column <code>is_late</code>. This column is equal to <code>model_data.arr_delay &gt; 0</code>.</li>
<li>Convert this column to an integer column so that you can use it in your model and name it <code>label</code> (this is the default name for the response variable in Spark's machine learning routines).</li>
<li>Filter out missing values (this has been done for you).</li>
</ul>

In [41]:
# Create is_late
model_data = model_data.withColumn("is_late", model_data.arr_delay > 0)

# Convert to an integer
model_data = model_data.withColumn("label", model_data.is_late.cast('integer'))

# Remove missing values
model_data = model_data.filter("arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and plane_year is not NULL")

## Strings and factors

<div class=""><p>As you know, Spark requires numeric data for modeling. So far this hasn't been an issue; even boolean columns can easily be converted to integers without any trouble. But you'll also be using the airline and the plane's destination as features in your model. These are coded as strings and there isn't any obvious way to convert them to a numeric data type.</p>
<p>Fortunately, PySpark has functions for handling this built into the <code>pyspark.ml.features</code> submodule. You can create what are called 'one-hot vectors' to represent the carrier and the destination of each flight. A <em>one-hot vector</em> is a way of representing a categorical feature where every observation has a vector in which all elements are zero except for at most one element, which has a value of one (1).</p>
<p>Each element in the vector corresponds to a level of the feature, so it's possible to tell what the right level is by seeing which element of the vector is equal to one (1).</p>
<p>The first step to encoding your categorical feature is to create a <code>StringIndexer</code>. Members of this class are <code>Estimator</code>s that take a DataFrame with a column of strings and map each unique string to a number. Then, the <code>Estimator</code> returns a <code>Transformer</code> that takes a DataFrame, attaches the mapping to it as metadata, and returns a new DataFrame with a numeric column corresponding to the string column.</p>
<p>The second step is to encode this numeric column as a one-hot vector using a <code>OneHotEncoder</code>. This works exactly the same way as the <code>StringIndexer</code> by creating an <code>Estimator</code> and then a <code>Transformer</code>. The end result is a column that encodes your categorical feature as a vector that's suitable for machine learning routines!</p>
<p>This may seem complicated, but don't worry! All you have to remember is that you need to create a <code>StringIndexer</code> and a <code>OneHotEncoder</code>, and the <code>Pipeline</code> will take care of the rest.</p>
<hr>
<p>Why do you have to encode a categorical feature as a one-hot vector?</p></div>

<pre>
Possible Answers
It makes fitting the model faster.
<b>Spark can only model numeric features.</b>
For compatibility with scikit-learn.
</pre>

**You remembered that Spark can only model numeric features.**

## Carrier

<div class=""><p>In this exercise you'll create a <code>StringIndexer</code> and a <code>OneHotEncoder</code> to code the <code>carrier</code> column. To do this, you'll call the class constructors with the arguments <code>inputCol</code> and <code>outputCol</code>.</p>
<p>The <code>inputCol</code> is the name of the column you want to index or encode, and the <code>outputCol</code> is the name of the new column that the <code>Transformer</code> should create.</p></div>

In [45]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

Instructions
<ul>
<li>Create a <code>StringIndexer</code> called <code>carr_indexer</code> by calling <code>StringIndexer()</code> with <code>inputCol="carrier"</code> and <code>outputCol="carrier_index"</code>.</li>
<li>Create a <code>OneHotEncoder</code> called <code>carr_encoder</code> by calling <code>OneHotEncoder()</code> with <code>inputCol="carrier_index"</code> and <code>outputCol="carrier_fact"</code>.</li>
</ul>

In [46]:
# Create a StringIndexer
carr_indexer = StringIndexer(inputCol="carrier", outputCol="carrier_index")

# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol="carrier_index", outputCol="carrier_fact")

## Destination

<p>Now you'll encode the <code>dest</code> column just like you did in the previous exercise.</p>

Instructions
<ul>
<li>Create a <code>StringIndexer</code> called <code>dest_indexer</code> by calling <code>StringIndexer()</code> with <code>inputCol="dest"</code> and <code>outputCol="dest_index"</code>.</li>
<li>Create a <code>OneHotEncoder</code> called <code>dest_encoder</code> by calling <code>OneHotEncoder()</code> with <code>inputCol="dest_index"</code> and <code>outputCol="dest_fact"</code>.</li>
</ul>

In [50]:
# Create a StringIndexer
dest_indexer = StringIndexer(inputCol="dest", outputCol="dest_index")

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

## Assemble a vector

<div class=""><p>The last step in the <code>Pipeline</code> is to combine all of the columns containing our features into a single column. This has to be done before modeling can take place because every Spark modeling routine expects the data to be in this form. You can do this by storing each of the values from a column as an entry in a vector. Then, from the model's point of view, every observation is a vector that contains all of the information about it and a label that tells the modeler what value that observation corresponds to.</p>
<p>Because of this, the <code>pyspark.ml.feature</code> submodule contains a class called <code>VectorAssembler</code>. This <code>Transformer</code> takes all of the columns you specify and combines them into a new vector column.</p></div>

In [52]:
from pyspark.ml.feature import VectorAssembler

Instructions
<li>Create a <code>VectorAssembler</code> by calling <code>VectorAssembler()</code> with the <code>inputCols</code> names as a list and the <code>outputCol</code> name <code>"features"</code>.<ul>
<li>The list of columns should be <code>["month", "air_time", "carrier_fact", "dest_fact", "plane_age"]</code>.</li></ul></li>

In [53]:
# Make a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol="features")

## Create the pipeline

<div class=""><p>You're finally ready to create a <code>Pipeline</code>!</p>
<p><code>Pipeline</code> is a class in the <code>pyspark.ml</code> module that combines all the <code>Estimators</code> and <code>Transformers</code> that you've already created. This lets you reuse the same modeling process over and over again by wrapping it up in one simple object. Neat, right?</p></div>

Instructions
<ul>
<li>Import <code>Pipeline</code> from <code>pyspark.ml</code>.</li>
<li>Call the <code>Pipeline()</code> constructor with the keyword argument <code>stages</code> to create a <code>Pipeline</code> called <code>flights_pipe</code>.<ul>
<li><code>stages</code> should be a list holding all the stages you want your data to go through in the pipeline. Here this is just: <code>[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler]</code></li></ul></li>
</ul>

In [55]:
# Import Pipeline
from pyspark.ml import Pipeline

# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

## Test vs Train

<div class=""><p>After you've cleaned your data and gotten it ready for modeling, one of the most important steps is to split the data into a <em>test set</em> and a <em>train set</em>. After that, don't touch your test data until you think you have a good model! As you're building models and forming hypotheses, you can test them on your training data to get an idea of their performance.</p>
<p>Once you've got your favorite model, you can see how well it predicts the new data in your test set. This never-before-seen data will give you a much more realistic idea of your model's performance in the real world when you're trying to predict or classify new data.</p>
<p>In Spark it's important to make sure you split the data <strong>after</strong> all the transformations. This is because operations like <code>StringIndexer</code> don't always produce the same index even when given the same list of strings.</p>
<hr>
<p>Why is it important to use a test set in model evaluation?</p></div>

<pre>
Possible Answers
Evaluating your model improves its accuracy.
<b>By evaluating your model with a test set you can get a good idea of performance on new data.</b>
Using a test set lets you check your code for errors.
</pre>

**A test set approximates the 'real world error' of your model.**

## Transform the data

<p>Hooray, now you're finally ready to pass your data through the <code>Pipeline</code> you created!</p>

Instructions
<li>Create the DataFrame <code>piped_data</code> by calling the <code>Pipeline</code> methods <code>.fit()</code> and <code>.transform()</code> in a chain. Both of these methods take <code>model_data</code> as their only argument.</li>

In [57]:
# Fit and transform the data
piped_data = flights_pipe.fit(model_data).transform(model_data)

## Split the data

<p>Now that you've done all your manipulations, the last step before modeling is to split the data!</p>

Instructions
<li>Use the DataFrame method <code>.randomSplit()</code> to split <code>piped_data</code> into two pieces, <code>training</code> with 60% of the data, and <code>test</code> with 40% of the data by passing the list <code>[.6, .4]</code> to the <code>.randomSplit()</code> method.</li>

In [59]:
# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])